In [32]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def gestor():
    start_time = time.time()  # Inicia el contador de tiempo
    access_token = 'EAAW2PQ0k6ZC8BOwxn9MltQMroQ54Ysd80WOSPVUe5iwwsgFb7J04rFlZANKJ38ZAchVas2ne6PbSDB9beO1wS7OZAwzRkTfKvh0EzNzbhsy30u9JAnSxgOxjBYcmT5jKNtjgNsgB8zOKEDYdWXUh8sQsVXtjLaGzS3BP31ZCrKPrKUknLXwazZBwZDZD'
    add_account_interes = 'act_203381180272654'
    campaing_ids = ['120208378118300095', '120208242119730095', '120208146785590095', '120208144837520095',
                '120208060828230095', '120208056585110095', '120207955015150095', '120207951304320095']
    with ThreadPoolExecutor() as executor:
        # Obtener ads en paralelo
        future_to_campaing_id = {executor.submit(obtenerAdds, campaing_id, access_token): campaing_id for campaing_id in campaing_ids}
        adds_results = {future_to_campaing_id[future]: future.result() for future in as_completed(future_to_campaing_id)}
    
    all_leads = {"dnis": [], "correos": [], "nleads": 0}
    
    with ThreadPoolExecutor() as executor:
        # Obtener leads en paralelo
        future_to_adds = {executor.submit(obtenerLeads, adds, access_token): adds for adds in adds_results.values()}
        leads_results = [future.result() for future in as_completed(future_to_adds)]
    
    for leads in leads_results:
        all_leads["dnis"].extend(leads["dnis"])
        all_leads["correos"].extend(leads["correos"])
        all_leads["nleads"] += leads["nleads"]
    
    print("All leads: ", all_leads)
    end_time = time.time()  # Finaliza el contador de tiempo para el gestor
    print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")

def obtenerAdds(campaing_id, access_token):
    url = f'https://graph.facebook.com/v19.0/{campaing_id}/ads?fields=id,name,creative'
    # Parámetros de la consulta
    params = {
        'fields': 'id,name',
        'limit': 100,
        'access_token': access_token
    }
    adds = []
    response = requests.get(url, params=params)
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        for add in data['data']:
            adds.append(add['id'])
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
    return adds

def obtenerLeads(adds, access_token):
    leads = {"dnis": [], "correos": [], "nleads": 0}
    for add in adds:
        url = f'https://graph.facebook.com/v20.0/{add}/leads'
        params = {
            'limit': 2000,
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            data = response.json()            
            for lead in data['data']:
                dni = None
                correo = None
                contador = 0
                for field in lead['field_data']:
                    if contador == 2:
                        break
                    if field['name'] == 'dni':
                        leads["dnis"].append(field['values'][0])
                        contador += 1
                    elif field['name'] == 'email':
                        leads["correos"].append(field['values'][0])
                        contador += 1
        else:
            print(f"Error: {response.status_code}")
            print(response.json())
        leads["nleads"] = len(leads["dnis"])
    return leads

gestor()


All leads:  {'dnis': ['40870570', '70324646', '45231617', '47522502', '75677194', '71257956', '70341752', '22079326', '43970668', '47621347', '18889464', '76407470', '10771402', '41802647', '40247294', '41390707', '44879886', '70864000', '43975290', '47160820', '48310027', '74499736', '46594410', '72937615', '74870437', '70454032', '47566707', '44327651', '40155856', '71210125', '41452930', '71397840', '46528497', '75654842', '72793671', '09358853', '48332651', '72203316', '15428596', '48292634', '72772914', '72948725', '43417605', '40687207', '74575703', '47801313', '46860027', '71231553', '72413006', '48414478', '40449796', '41838035', '44941772', '63963696', '46700279', '73380309', '21556695', '09588209', '47281812', '72700210', '74120064', '77283453', '09368979', '40362675', '12345678', '72323388', '41971387', '42906913', '74302896', '42002961', '29719577', '70445454', '40558711', '76266319', '41633797', '71229483', '26646228', '20426385', '06797519', '44974252', '09633330', '76039

In [ ]:
    campaing_ids = ['120208378118300095', '120208242119730095', '120208146785590095', '120208144837520095',
                    '120208060828230095', '120208056585110095', '120207955015150095', '120207951304320095']

In [4]:
import time
import requests
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

lock = threading.Lock()
def gestor():
    start_time = time.time()  # Inicia el contador de tiempo
    access_token = 'EAAW2PQ0k6ZC8BOwxn9MltQMroQ54Ysd80WOSPVUe5iwwsgFb7J04rFlZANKJ38ZAchVas2ne6PbSDB9beO1wS7OZAwzRkTfKvh0EzNzbhsy30u9JAnSxgOxjBYcmT5jKNtjgNsgB8zOKEDYdWXUh8sQsVXtjLaGzS3BP31ZCrKPrKUknLXwazZBwZDZD'
    campaing_ids = ['120208378118300095', '120208242119730095', '120208146785590095', '120208144837520095']
    with ThreadPoolExecutor() as executor:
        # Obtener ads en paralelo
        future_to_campaing_id = {executor.submit(obtenerAdds, campaing_id, access_token): campaing_id for campaing_id in campaing_ids}
        adds_results = {future_to_campaing_id[future]: future.result() for future in as_completed(future_to_campaing_id)}

    all_leads = {campaing_id: {"dnis": [], "correos": [], "nleads": 0} for campaing_id in campaing_ids}

    with ThreadPoolExecutor() as executor:
        # Obtener leads en paralelo
        future_to_campaing_id = {executor.submit(obtenerLeads, adds_results[campaing_id], access_token): campaing_id for campaing_id in campaing_ids}
        for future in as_completed(future_to_campaing_id):
            campaing_id = future_to_campaing_id[future]
            leads = future.result()
            with lock:
                # Se adquiere el bloqueo antes de modificar all_leads
                all_leads[campaing_id]["dnis"].extend(leads["dnis"])
                all_leads[campaing_id]["correos"].extend(leads["correos"])
                all_leads[campaing_id]["nleads"] += leads["nleads"]

    for campaing_id, leads in all_leads.items():
        print(f"Campaign ID: {campaing_id}")
        print("Leads: ", leads)

    end_time = time.time()  # Finaliza el contador de tiempo para el gestor
    print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")

def obtenerAdds(campaing_id, access_token):
    url = f'https://graph.facebook.com/v19.0/{campaing_id}/ads?fields=id,name,creative'
    params = {
        'fields': 'id,name',
        'limit': 100,
        'access_token': access_token
    }
    adds = []
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        for add in data['data']:
            adds.append(add['id'])
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
    return adds

def obtenerLeads(adds, access_token):
    leads = {"dnis": [], "correos": [], "nleads": 0}
    for add in adds:
        url = f'https://graph.facebook.com/v20.0/{add}/leads'
        params = {
            'limit': 2000,
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()            
            for lead in data['data']:
                contador = 0
                for field in lead['field_data']:
                    if contador == 2:
                        break
                    if field['name'] == 'dni':
                        leads["dnis"].append(field['values'][0])
                        contador += 1
                    elif field['name'] == 'email':
                        leads["correos"].append(field['values'][0])
                        contador += 1
        else:
            print(f"Error: {response.status_code}")
            print(response.json())
        leads["nleads"] = len(leads["dnis"])
    return leads

gestor()
    

Campaign ID: 120208378118300095
Leads:  {'dnis': ['47235169', '47416378', '45234693', '10662842', '40388361', '10823205', '25836576', '47151351', '45618056', '09540661', '42123626', '46951076', '73520662', '60022375', '47435990', '47485260', '43520326'], 'correos': ['a20194852@pucp.edu.pe', 'Yasmani_mcj@hotmail.com', 'luisa_alza@hotmail.com', 'Ymejia@iqmeh.com.pe', 'alex.vilchezc@pucp.pe', 'cberaunsilva@gmail.com', 'mazanett@gmail.com', 'roxy07.vera@gmail.com', 'jfortonc@gmail.com', 'kperez@grupoandina.com.pe', 'ccinthya.auccacusi@gmail.com', 'm.marquina.j@gmail.com', 'ar.ramosramirez@gmail.com', 'haguirreart@gmail.con', 'angelalopezni@gmail.com', 'yesenia.calderon.dominguez@gmail.com', 'cclaudiapoma@gmail.com'], 'nleads': 17}
Campaign ID: 120208242119730095
Leads:  {'dnis': ['43544576', '76607455', '74088375', '48010855', '44696395', '71651857', '75504489', '71886494', '70153910', '70795554', '01375868', '72687004', '41957864', '71721657', '70267447', '72042080', '26695645', '00000001

In [7]:
import requests

def gestor():
    start_time = time.time()  # Inicia el contador de tiempo
    access_token = 'EAAW2PQ0k6ZC8BOwxn9MltQMroQ54Ysd80WOSPVUe5iwwsgFb7J04rFlZANKJ38ZAchVas2ne6PbSDB9beO1wS7OZAwzRkTfKvh0EzNzbhsy30u9JAnSxgOxjBYcmT5jKNtjgNsgB8zOKEDYdWXUh8sQsVXtjLaGzS3BP31ZCrKPrKUknLXwazZBwZDZD'
    add_account_interes='act_203381180272654'
    campaing_ids = ['120208378118300095', '120208242119730095', '120208146785590095', '120208144837520095']
    for campaing_id in campaing_ids:
        adds=obtenerAdds(campaing_id,access_token)
        leads=obtenerLeads(adds,access_token)
        print("campaing_id:" ,  campaing_id)
        print("adds: ",adds)
        print("leads: ",leads)
    end_time = time.time()  # Finaliza el contador de tiempo para el gestor
    print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")
    
def obtenerAdds(campaing_id,access_token):
    url = f'https://graph.facebook.com/v19.0/{campaing_id}/ads?fields=id,name,creative'
    # Parámetros de la consulta
    params = {
        'fields': 'id,name',
        'limit': 100,
        'access_token': access_token
    }
    adds=[]
    response = requests.get(url, params=params)
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        for add in data['data']:
            adds.append(add['id'])
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
    return adds

def obtenerLeads(adds,access_token):
    leads={"dnis":[],"correos":[],"nleads":0}
    for add in adds:
        url = f'https://graph.facebook.com/v20.0/{add}/leads'
        params = {
            'limit': 2000,
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        # Verificar si la solicitud fue exitosa
        if response.status_code == 200:
            data = response.json()            
            for lead in data['data']:
                dni = None
                correo = None
                contador=0
                for field in lead['field_data']:
                    if contador==2:
                        break
                    if field['name'] == 'dni':
                        leads["dnis"].append(field['values'][0])
                        contador+=1
                    elif field['name'] == 'email':
                        leads["correos"].append(field['values'][0])
                        contador+=1      
        else:
            print(f"Error: {response.status_code}")
            print(response.json())
        leads["nleads"]=len(leads["dnis"])
    return leads

gestor()

campaing_id: 120208378118300095
adds:  ['120208378926220095', '120208379189800095', '120208379189790095', '120208379189780095', '120208378118290095', '120208378880890095']
leads:  {'dnis': ['47235169', '47416378', '45234693', '10662842', '40388361', '10823205', '25836576', '47151351', '45618056', '09540661', '42123626', '46951076', '73520662', '60022375', '47435990', '47485260', '43520326'], 'correos': ['a20194852@pucp.edu.pe', 'Yasmani_mcj@hotmail.com', 'luisa_alza@hotmail.com', 'Ymejia@iqmeh.com.pe', 'alex.vilchezc@pucp.pe', 'cberaunsilva@gmail.com', 'mazanett@gmail.com', 'roxy07.vera@gmail.com', 'jfortonc@gmail.com', 'kperez@grupoandina.com.pe', 'ccinthya.auccacusi@gmail.com', 'm.marquina.j@gmail.com', 'ar.ramosramirez@gmail.com', 'haguirreart@gmail.con', 'angelalopezni@gmail.com', 'yesenia.calderon.dominguez@gmail.com', 'cclaudiapoma@gmail.com'], 'nleads': 17}
campaing_id: 120208242119730095
adds:  ['120208242119770095', '120208252000920095', '120208251971230095', '1202082520009100

In [25]:
import requests

# Tu token de acceso
access_token = 'EAAW2PQ0k6ZC8BOwxn9MltQMroQ54Ysd80WOSPVUe5iwwsgFb7J04rFlZANKJ38ZAchVas2ne6PbSDB9beO1wS7OZAwzRkTfKvh0EzNzbhsy30u9JAnSxgOxjBYcmT5jKNtjgNsgB8zOKEDYdWXUh8sQsVXtjLaGzS3BP31ZCrKPrKUknLXwazZBwZDZD'

# Endpoint de la API
url = 'https://graph.facebook.com/v16.0/120208149806280095/leads'

# Parámetros de la consulta
params = {
    'fields': 'field_data',
    'access_token': access_token,
    'limit': 20
}

# Realizar la solicitud GET
response = requests.get(url, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")
    print(response.json())


{'data': [{'field_data': [{'name': '¿autorizo_a_centrum_pucp_a_enviarme_información_sobre_sus_programas_académicos?_al_seleccionar_si,_aceptas_recibir_información_de_centrum_pucp_sobre_este_y_otros_programas_relevantes_afines_al_seleccionado,_vía_email_/_llamada_/_whatsapp_/_sms._en_caso_de_haber_brindado_su_autorización,_y_ya_no_desee_continuar_recibiendo_información,_podrá_escribirnos_a_centruminformes@pucp.edu.pe_para_realizar_el_procedimiento_pertinente_de_desafiliación.', 'values': ['true']}, {'name': 'last_name', 'values': ['Samata Ancco']}, {'name': 'phone_number', 'values': ['+51969746149']}, {'name': 'first_name', 'values': ['Gissel Emma']}, {'name': 'email', 'values': ['gsamata08@gmail.com']}, {'name': 'dni', 'values': ['74827307']}], 'id': '7641470595966730'}, {'field_data': [{'name': 'email', 'values': ['mlc2209@yahoo.com']}, {'name': 'dni', 'values': ['07484406']}, {'name': 'last_name', 'values': ['Lopera']}, {'name': 'phone_number', 'values': ['+51959371752']}, {'name': '